The task of the competition was to find the best model which has maximum AUC. I used EvalMl and H2O AI platform for selecting the best model and parameters for training my model. Hope U guys enjoy and learn something insightful to you.
Thanks. Please Upvote :D .

# Installing and Importing relevant libraries

In [ ]:
!pip install evalml
!pip install h2o

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from evalml.automl import AutoMLSearch
import pandas as pd
import numpy as np
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
#from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
df = pd.read_csv('../input/jobathon-analytics-vidhya/train.csv')
test = pd.read_csv('../input/jobathon-analytics-vidhya/test.csv')

In [ ]:
#Since ID is not to be used|
df = df.set_index('ID')
test = test.set_index('ID')

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(['Response'],axis=1).copy(),df['Response'],test_size=0.2,random_state=42)

In [ ]:
from evalml.automl import AutoMLSearch
automl = AutoMLSearch(X_train=x_train, y_train=y_train, problem_type='binary')
automl.search()

In [ ]:
!pip uninstall pandas -y
!pip install pandas

In [ ]:
automl.rankings

In [ ]:
automl.describe_pipeline(2) #We can then follow the steps and build our model or directly use it

In [ ]:
#instantiate both packages to use
encoder = OrdinalEncoder()
imputer = IterativeImputer(ExtraTreesRegressor())
# create a list of categorical columns to iterate over
cat_cols_df = df.columns
cat_cols_test = test.columns
def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

#create a for loop to iterate through each column in the data
for columns in cat_cols_df:
    encode(df[columns])
for columns in cat_cols_test:
    encode(test[columns])

In [ ]:
test.info(),df.info()

In [ ]:
imputer.fit_transform(df)

In [ ]:
encode_data = pd.DataFrame(np.round(imputer.transform(df)),columns = df.columns)

In [ ]:
encode_data

In [ ]:
test = pd.DataFrame(np.round(imputer.fit_transform(test)),columns = test.columns)

In [ ]:
test

In [ ]:
test.to_csv('my_new_test.csv')
encode_data.to_csv('my_new_data.csv')#making a backup

In [ ]:
import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

In [ ]:
train = h2o.H2OFrame(encode_data)
train.head()

In [ ]:
test = h2o.import_file('./my_new_test.csv')
# test = test.drop(['C1','ID'])
test.head()

In [ ]:
from h2o.estimators import H2OTargetEncoderEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators import H2OStackedEnsembleEstimator,H2OXGBoostEstimator
from h2o.grid.grid_search import H2OGridSearch

In [ ]:
train['Response'] = train['Response'].asfactor()

In [ ]:
x = train.columns
y = 'Response'
x.remove(y)
x

In [ ]:
aml = H2OAutoML(max_models=None, seed=42, max_runtime_secs=300,exploitation_ratio=0.2,nfolds=5,balance_classes=True,stopping_metric='AUC')
aml.train(x=x, y=y, training_frame=train)
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
preds = aml.predict(test)

In [ ]:
test_ = h2o.H2OFrame(pd.read_csv('../input/jobathon-analytics-vidhya/test.csv'))

In [ ]:
preds = preds.cbind(test_['ID'])

In [ ]:
preds = preds.drop(['predict','p0'])

In [ ]:
preds['Response'] = preds['p1']
preds = preds.drop(['p1'])

In [ ]:
preds


In [ ]:
h2o.export_file(preds,path='./submission_f.csv',force=True)

In [ ]:
len(test_)

In [ ]:
preds = pd.read_csv('./submission_f.csv')

In [ ]:
preds

**Please Upvote If you Liked it!!!**